<span style="color:yellow">LIBRAIRIES ET FONCTIONS</span>

In [1]:
import pandas as pd
import xgboost as xgb
import os
import pickle
import flask
from flask import request, jsonify
import numpy as np
import json
import requests

<span style="color:yellow">CHARGEMENT DU MODÈLE</span>

Flask est un micro-framework web léger et flexible pour le développement d'applications web en Python. Il a été créé par Armin Ronacher et est distribué sous licence BSD. Flask est considéré comme un micro-framework car il ne nécessite que les packages Python de base et dispose d'un noyau minimaliste, mais il est facilement extensible grâce à ses nombreuses extensions tierces.

Voici quelques caractéristiques clés de Flask :

Léger et modulaire : Flask a un noyau minimal et est facile à configurer. Il vous permet de commencer rapidement avec un projet web.

Extensible : Flask peut être étendu avec des extensions tierces pour ajouter des fonctionnalités supplémentaires comme la gestion des bases de données, la validation des formulaires, la mise en cache, etc.

Débogage intégré : Flask intègre un serveur de développement et un débogueur pour faciliter le développement et le débogage.

Routage URL : Flask fournit un système de routage URL simple mais puissant pour mapper les URL aux fonctions Python.

Rendu de modèles : Flask prend en charge les moteurs de rendu de modèles comme Jinja2, ce qui facilite la génération de contenu dynamique.

Requêtes et réponses : Flask facilite la manipulation des requêtes HTTP et la construction de réponses.

Intégration avec les bibliothèques Python existantes : Flask peut facilement s'intégrer avec d'autres bibliothèques Python populaires comme SQLAlchemy pour la gestion des bases de données, Werkzeug pour les utilitaires WSGI, etc.

Flask est souvent utilisé pour créer des applications web simples, des API RESTful, des services web ou des prototypes rapides. Il est apprécié pour sa simplicité, sa flexibilité et sa facilité d'apprentissage. Cependant, pour des projets plus importants ou complexes, des frameworks plus complets comme Django peuvent être préférés.

In [ ]:
import os
import pickle
import numpy as np
import xgboost as xgb

# Chemin vers le dossier du projet Poetry
project_dir = '/home/tamara-daniel-tricot/Bureau/0_Projets_ParcoursDataScientist/P7/Projet_7/'

# Charger le modèle XGBoost
model_path = os.path.join(project_dir, 'mon_modele.json')
print(f"Chargement du modèle depuis : {model_path}")
model = xgb.Booster(model_file=model_path)

# Charger le seuil de décision
threshold_path = os.path.join(project_dir, 'seuil_decision.pkl')
print(f"Chargement du seuil de décision depuis : {threshold_path}")
with open(threshold_path, 'rb') as file:
    threshold = pickle.load(file)
    print(f"Seuil de décision chargé : {threshold}")

# Fonction pour faire des prédictions avec le modèle chargé
def predict(X):
    print("Données d'entrée X :")
    print(X)
    print(f"Nombre d'instances : {len(X)}")
    print(f"Nombre de colonnes : {len(X[0])}")
    print("Types de données des colonnes :")
    for i, instance in enumerate(X[:5]):  # Affiche les 5 premières instances
        print(f"Instance {i} : {[type(val) for val in instance]}")

    dmatrix = xgb.DMatrix(np.array(X), missing=np.inf)
    prediction_proba = model.predict(dmatrix)
    print("Types de données des probabilités de prédiction :")
    print(type(prediction_proba))
    if isinstance(prediction_proba, np.ndarray):
        prediction_proba = prediction_proba.tolist()
        print("Conversion des probabilités de prédiction en liste Python")
    else:
        print("Les probabilités de prédiction sont déjà une liste Python")

    prediction = [1 if proba >= threshold else 0 for proba in prediction_proba]
    print("Types de données des prédictions :")
    print(type(prediction))
    if isinstance(prediction, np.ndarray):
        prediction = prediction.tolist()
        print("Conversion des prédictions en liste Python")
    else:
        print("Les prédictions sont déjà une liste Python")

    return prediction, prediction_proba

<span style="color:yellow">DÉPLOIEMENT DE L'API Flask : pour démarrer le serveur Flask et pouvoir envoyer des requêtes à l'API

In [ ]:
import flask
from flask_debugtoolbar import DebugToolbarExtension
import importlib.metadata
import numpy as np
import os
import json
import pickle
import xgboost as xgb
from modele_xgboost import predict
import logging
from logging.handlers import RotatingFileHandler

# Configurer les logs
file_handler = RotatingFileHandler('flask_app.log', maxBytes=16384, backupCount=20)
file_formatter = logging.Formatter('%(asctime)s %(levelname)s: %(message)s [in %(pathname)s:%(lineno)d]')
file_handler.setFormatter(file_formatter)
file_handler.setLevel(logging.INFO)

app = flask.Flask(__name__)
app.logger.addHandler(file_handler)
app.logger.setLevel(logging.INFO)

flask_version = importlib.metadata.version('flask')
print(f"Version de Flask : {flask_version}")

@app.route('/')
def index():
    return 'Bienvenue sur mon API Flask !'

@app.route('/predict', methods=['GET', 'POST'])
def predict_endpoint():
    app.logger.info(f"Requête reçue à l'URL : {flask.request.url}")
    data = flask.request.get_json()['data']
    X = np.array([np.array(instance) for instance in data])

    try:         
        prediction, prediction_proba = predict(X)
        app.logger.info('Prédiction effectuée avec succès')
        return flask.jsonify({
            'prediction': prediction,
            'prediction_proba': prediction_proba
        })
    except Exception as e:
        app.logger.error(f'Erreur lors de la prédiction : {e}')
        return flask.jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    print("Démarrage du serveur Flask...")
    app.run(debug=True)
    print("Serveur Flask démarré")

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/tamara-daniel-tricot/Bureau/0_Projets_ParcoursDataScientist/P7/Projet_7/.venv/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/tamara-daniel-tricot/Bureau/0_Projets_ParcoursDataScientist/P7/Projet_7/.venv/lib/python3.12/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/home/tamara-daniel-tricot/Bureau/0_Projets_ParcoursDataScientist/P7/Projet_7/.venv/lib/python3.12/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/tamara-daniel-tricot/Bureau/0_Projets_ParcoursDataScientist/P7/Projet_7/.venv/lib/python3.12/site-package

SystemExit: 1

/home/tamara-daniel-tricot/Bureau/0_Projets_ParcoursDataScientist/P7/Projet_7/.venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


<span style="color:yellow">INTERACTION AVEC L'API : pour envoyer un jeu de données au modèle xgboost chargé sur l'API

In [ ]:
import pandas as pd
import requests
import numpy as np

# Charger les données de test depuis un fichier CSV
df_final_test = pd.read_csv('/home/tamara-daniel-tricot/Bureau/0_Projets_ParcoursDataScientist/P7/Projet_7/df_final_test.csv')

# Prendre les 5 premières lignes de df_final_test
df_final_test_sample = df_final_test.head(100000)

# Remplacer les valeurs NaN et infinies par une valeur valide (par exemple, 0)
df_final_test_sample = df_final_test_sample.replace([np.inf, -np.inf, np.nan], 0)

# Convertir le DataFrame en un tableau NumPy
X_test_sample = df_final_test_sample.values

# Créer un dictionnaire avec la clé "data" et la valeur étant le tableau NumPy
data = {"data": X_test_sample.tolist()}

# Envoyer une requête POST à l'API Flask
response = requests.post('http://localhost:5000/predict', json=data)

# Récupérer les prédictions et les probabilités à partir de la réponse de l'API
if response.status_code == 200:
    predictions = response.json()
    print("Prédictions :", predictions['prediction'])
    print("Probabilités :", predictions['prediction_proba'])
else:
    print(f"Erreur : {response.status_code}")